In [2]:
import pandas as pd
import numpy as np

raw_data = pd.read_parquet('/Users/lucazosso/Desktop/IE_Course/Term_3/Algorithmic_Trading/ie_mbd_sept23/sim/Raw.parquet')
raw_data.head()

,Time,Type,OrderID,Size,Price,Ask Price 1,Ask Size 1,Bid Price 1,Bid Size 1,Ask Price 2,...,Ask Price 9,Ask Size 9,Bid Price 9,Bid Size 9,Ask Price 10,Ask Size 10,Bid Price 10,Bid Size 10,midprice,Calc_direction
0,34200.004241,1,16113575,18,5853300,5859400,200,5853300,18,5859800,...,5876500,1160,5843800,200,5879000,500,5842700,300,5856350.0,Sell
1,34200.004261,1,16113584,18,5853200,5859400,200,5853300,18,5859800,...,5876500,1160,5845300,300,5879000,500,5843800,200,5856350.0,Sell
2,34200.004447,1,16113594,18,5853100,5859400,200,5853300,18,5859800,...,5876500,1160,5846500,300,5879000,500,5845300,300,5856350.0,Sell
3,34200.025552,1,16120456,18,5859100,5859100,18,5853300,18,5859400,...,5873900,100,5846500,300,5876500,1160,5845300,300,5856200.0,Buy
4,34200.025580,1,16120480,18,5859200,5859100,18,5853300,18,5859200,...,5871000,10,5846500,300,5873900,100,5845300,300,5856200.0,Buy


In [7]:
raw_data.shape

(430266, 47)

# TBD

In [6]:
# Group Data by Time and Type and Calc_direction

processed_data = raw_data.groupby(['Time', 'Type', 'Calc_direction']).agg({'Price': 'mean', 
                                                                           'Size': 'sum', 
                                                                           'midprice': 'mean'}).reset_index()

processed_data.head()



# Chunking Data into batches of 1 min to reduce noises. The data is already sorted by time.

,Time,Type,Calc_direction,Price,Size,midprice
0,34200.004241,1,Sell,5853300.0,18,5856350.0
1,34200.004261,1,Sell,5853200.0,18,5856350.0
2,34200.004447,1,Sell,5853100.0,18,5856350.0
3,34200.025552,1,Buy,5859100.0,18,5856200.0
4,34200.025580,1,Buy,5859200.0,18,5856200.0


In [9]:
# Do a OHLC for each minute batch by Type and Calc_direction
def generate_ohlc_tables(data, time_interval: str = '1min') -> dict:
    # Create a dictionary to store the OHLC dataframes
    ohlc_tables = {}
    
    # Define the types and directions
    types = [1, 2, 3, 4]
    directions = ['Sell', 'Buy']

    # Generate the OHLC tables
    for type_ in types:
        for direction in directions:
            df = data[(data['Type'] == type_) & (data['Calc_direction'] == direction)]
            ohlc_table = df.groupby(pd.Grouper(key='Time', freq=time_interval)).agg({'Price': 'ohlc', 'Size': 'sum'}).reset_index()
            ohlc_tables[f'type_{type_}_{direction}'] = ohlc_table

    return ohlc_tables


In [11]:
ohlc_tables = generate_ohlc_tables(processed_data, time_interval='1min')

ohlc_tables.keys()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'